In [ ]:
import matplotlib.pyplot as plt
import dill
import sympy
from tqdm import tqdm
from scipy.sparse import lil_matrix, csc_matrix
from scipy.sparse.linalg import spsolve

### Load precomputed integrals

In [ ]:
lagrange_linear_poisson_matrix = dill.load(
    open("../calculations/lagrange_linear_poisson_matrix", "rb")
)
lagrange_linear_poisson_right_values = dill.load(
    open("../calculations/lagrange_linear_poisson_right_values", "rb")
)

### Initialize the domain

In [ ]:
import gmsh
import numpy as np
import matplotlib.pyplot as plt


gmsh.initialize()
gmsh.model.add("quad_mesh")

ms = 0.25
# Define points
point_1 = gmsh.model.geo.addPoint(0, 0, 0, ms)
point_2 = gmsh.model.geo.addPoint(1, 0, 0, ms)
point_3 = gmsh.model.geo.addPoint(0, 1, 0, ms)
point_4 = gmsh.model.geo.addPoint(-1, 0, 0, ms)
point_5 = gmsh.model.geo.addPoint(0, -1, 0, ms)

# Define circles
arc_1 = gmsh.model.geo.addCircleArc(point_2, point_1, point_3)
arc_2 = gmsh.model.geo.addCircleArc(point_3, point_1, point_4)
arc_3 = gmsh.model.geo.addCircleArc(point_4, point_1, point_5)
arc_4 = gmsh.model.geo.addCircleArc(point_5, point_1, point_2)

# Define line loop
circle = gmsh.model.geo.addCurveLoop([arc_1, arc_2, arc_3, arc_4])

# Define plane surface
plane = gmsh.model.geo.addPlaneSurface([circle])

gmsh.model.geo.synchronize()

ms = 100
gmsh.model.mesh.setTransfiniteCurve(arc_1, ms)
gmsh.model.mesh.setTransfiniteCurve(arc_2, ms)
gmsh.model.mesh.setTransfiniteCurve(arc_3, ms)
gmsh.model.mesh.setTransfiniteCurve(arc_4, ms)
gmsh.model.mesh.setTransfiniteSurface(plane)

physical_group_curves_tag = gmsh.model.addPhysicalGroup(1, [arc_1, arc_2, arc_3, arc_4], name="Boundary curves")
physical_group_surface_tag = gmsh.model.addPhysicalGroup(2, [plane], name="Surface")

gmsh.option.setNumber(name="Mesh.Smoothing", value=20)
gmsh.model.mesh.generate(2)
gmsh.fltk.run()

# Triangles

In [ ]:
boundary_node_tags, boundary_node_coords = gmsh.model.mesh.getNodesForPhysicalGroup(1, physical_group_curves_tag)
surface_node_tags, points = gmsh.model.mesh.getNodesForPhysicalGroup(2, physical_group_surface_tag)
points = points.reshape(-1, 3)[:, :-1]
boundary_node_tags = boundary_node_tags - 1

In [ ]:
element_types, element_tags, element_node_tags = gmsh.model.mesh.getElements()
triangles = [nodes for elem_type, nodes in zip(element_types, element_node_tags) if elem_type == 2][0]
triangles = triangles.reshape(-1, 3)
triangles = triangles - 1

In [ ]:
vertex_marker_is_boundary = np.zeros(surface_node_tags.shape[0])
vertex_marker_is_boundary[boundary_node_tags] = 1

In [ ]:
all_edges = triangles[:, [[0, 1], [1, 2], [0, 2]]].reshape(-1, 2)
all_edges = np.sort(all_edges, axis=-1)
all_edges, indices_1, indices, counts = np.unique(all_edges, axis=0, return_counts=True, return_inverse=True, return_index=True)
edge_marker_is_boundary = (counts == 1).astype(int)[:, None]

## Triangles to edges

In [ ]:
triangles_to_edges = indices.reshape(-1, 3)

## Vertices to edges

In [ ]:
vertices_to_edges = []

for i in range(points.shape[0]):
    vertices_to_edges.append([])

for eidx, edge in enumerate(all_edges):
    for pidx in edge:
        vertices_to_edges[pidx].append(eidx)

## Edges to edges

In [ ]:
edges_to_edges = []

for i in range(all_edges.shape[0]):
    edges_to_edges.append([])

for eidx, edge in enumerate(all_edges):
    
    edges_to_edges[eidx].extend(vertices_to_edges[edge[0]])
    edges_to_edges[eidx].extend(vertices_to_edges[edge[1]])
    edges_to_edges[eidx] = list(set(edges_to_edges[eidx]))
    edges_to_edges[eidx].remove(eidx)

## Edges to triangles

In [ ]:
edges_to_triangles = []

for i in range(all_edges.shape[0]):
    edges_to_triangles.append([])

for cidx, edge in enumerate(triangles_to_edges):
    
    for eidx in edge:
        edges_to_triangles[eidx].append(cidx)

## Vertices to triangles

In [ ]:
vertices_to_triangles = []

for i in range(points.shape[0]):
    vertices_to_triangles.append([])

for cidx, cell in enumerate(triangles):
    for pidx in cell:
        vertices_to_triangles[pidx].append(cidx)

## Triangles to triangles

In [ ]:
triangles_to_triangles = []

for i in range(triangles.shape[0]):
    triangles_to_triangles.append([])
    
for e2t in edges_to_triangles:
    
    if len(e2t) == 1:        
        triangles_to_triangles[e2t[0]].append(-1)
    else:
        triangles_to_triangles[e2t[0]].append(e2t[1])
        triangles_to_triangles[e2t[1]].append(e2t[0])

In [ ]:
x, y = sympy.symbols("x y")

In [ ]:
F = sympy.Float(1)
right_vals = np.zeros((points.shape[0]))
right_vals[:] = sympy.lambdify((x, y), F)(*points.T)

G = sympy.Float(0)
right_vals_G = np.zeros((points.shape[0]))
right_vals_G[:] = sympy.lambdify((x, y), G)(*points.T)

In [ ]:
batched_triangles = points[triangles]

In [ ]:
bilinear_from = lagrange_linear_poisson_matrix(batched_triangles[:, 0, 0],
                                    batched_triangles[:, 0, 1],
                                    batched_triangles[:, 1, 0],
                                    batched_triangles[:, 1, 1],
                                    batched_triangles[:, 2, 0],
                                    batched_triangles[:, 2, 1],
                                    ).transpose(2, 0, 1)

righ_hand_side = lagrange_linear_poisson_right_values(batched_triangles[:, 0, 0],
                                        batched_triangles[:, 0, 1],
                                        batched_triangles[:, 1, 0],
                                        batched_triangles[:, 1, 1],
                                        batched_triangles[:, 2, 0],
                                        batched_triangles[:, 2, 1],
                                        right_vals[triangles[:, 0]],
                                        right_vals[triangles[:, 1]],
                                        right_vals[triangles[:, 2]],
                                        ).squeeze().T

In [ ]:
b = np.zeros((points.shape[0]), dtype=np.float64)

idx, local_point, i = np.meshgrid(np.arange(triangles.shape[0]), np.arange(3), np.arange(3), indexing='ij')

row_indices = triangles[idx, local_point].ravel()
col_indices = triangles[idx, i].ravel()
values = bilinear_from[idx, local_point, i].ravel()

mask = (vertex_marker_is_boundary[triangles[idx, local_point]].ravel() == 0) & (vertex_marker_is_boundary[triangles[idx, i]].ravel() == 0)
m_mask = vertex_marker_is_boundary[triangles[idx, local_point]].ravel() == 1

np.subtract.at(b, col_indices[m_mask], right_vals_G[row_indices[m_mask]] * values[m_mask])

V = np.concatenate([np.ones(int(vertex_marker_is_boundary.sum())), values[mask]])
R = np.concatenate([np.where(vertex_marker_is_boundary == 1)[0], row_indices[mask]])
C = np.concatenate([np.where(vertex_marker_is_boundary == 1)[0], col_indices[mask]])

matrix = csc_matrix((V, (R, C)), shape=(points.shape[0], points.shape[0]), dtype=np.float64)


idx, local_point = np.meshgrid(np.arange(triangles.shape[0]), np.arange(3), indexing='ij')

row_indices = triangles[idx, local_point].ravel()
values = righ_hand_side[idx, local_point].ravel()
mask = vertex_marker_is_boundary[triangles[idx, local_point]].ravel() == 0
v_mask = vertex_marker_is_boundary == 1

np.subtract.at(b, row_indices[mask], values[mask])

b[v_mask] = right_vals_G[v_mask]

In [ ]:
F = spsolve(matrix, b)

In [ ]:
_x_ = points[:, 0]
_y_ = points[:, 1]

In [ ]:
import plotly
plot_data = [
    plotly.graph_objects.Mesh3d(
                                x=_x_, 
                                y=_y_,
                                z=F,
                                intensity=F
                                ),
    ]
fig = plotly.graph_objects.Figure(data=plot_data)
fig.update_layout()

In [ ]:
import scipy

In [ ]:
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(A=matrix, k=1000)

In [ ]:
eigenvectors = eigenvectors.T

In [ ]:
import plotly

eigh = eigenvectors[-411]
plot_data = [
    plotly.graph_objects.Mesh3d(
                                x=_x_, 
                                y=_y_,
                                z=eigh,
                                intensity=eigh
                                ),
    ]
fig = plotly.graph_objects.Figure(data=plot_data)
fig.update_layout()